<a href="https://colab.research.google.com/github/vasyafon/BayDS/blob/master/Copy_of_Modeling_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline



In [4]:
main_path = r'../../'

In [5]:
import sys
sys.path.append(main_path)
from BayDS import *

Using TensorFlow backend.


In [6]:
experiment_name = 'cat_boost_1'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [8]:
train = pd.read_pickle(f'{main_path}/Data/train_09457_with_additions.pkl')
test = pd.read_pickle(f'{main_path}/Data/test_09457_with_additions.pkl')


In [9]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT'], axis=1)
y = train.sort_values('TransactionDT')['isFraud'].astype(np.uint8)
test = test.sort_values('TransactionDT').drop(['TransactionDT'], axis=1)

X.drop('Date', axis=1, inplace=True)
X=X.astype(np.float32)
test.drop('Date', axis=1, inplace=True)
test=test.astype(np.float32)
del train
gc.collect()

48

In [10]:
# Setting model_folder
model_name = 'catboost-test'
model_folder = f'{main_learning_folder}/{model_name}'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [11]:
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=n_fold)
# folds = GroupKFold(n_splits=5)
# groups = pd.read_pickle('./groups.pkl').values

In [15]:
params = {
          'grow_policy':'Lossguide',
           'max_leaves':256,
           'od_type':'Iter',
#            'num_leaves': 256,
          'od_wait':200, 
#           'is_unbalance':True,
#           'feature_fraction':0.8,
          'max_leaves':256,
#           'verbosity':1,
#           'min_data_in_leaf': 106,
          'objective': 'Logloss',
          'learning_rate': 0.01,
#           'loss_function':'Logloss',
          'use_best_model':True,
          "logging_level": 'Verbose',
          'random_state': 47,
#           'bootstrap_type': 'Bayesian',
          'task_type': 'GPU',
         }
train_options = {
    "model_type":'cat',
    "params": params,
    'early_stopping_rounds': 200,
    'n_estimators': 3000,
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
    
}

In [16]:
import json
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)

In [17]:
print(gc.collect())
result_dict_lgb = train_model_classification(X=X, X_test=test, y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             plot_feature_importance=True,
                                             verbose=500, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
                                             n_jobs=-1, groups=None)

187
Fold 1 started at Tue Sep  3 23:01:18 2019
Fold 2 started at Tue Sep  3 23:06:18 2019
Fold 3 started at Tue Sep  3 23:10:40 2019
Fold 4 started at Tue Sep  3 23:15:35 2019
Fold 5 started at Tue Sep  3 23:20:33 2019
CV mean score: 0.6757, std: 0.0060.


In [18]:
import pickle
with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_lgb,f)
#     f.write(q)